In [16]:
filepath_common = 'proturn_output_'
strains = ['aj', 'balbc', 'c57', 'cej', 'dba', 'fvb']
exp = ['ctrl', 'iso'] 
hl = 'hl.out'
dat = 'hl-data_masked'
mask = [1,2,3,4]

In [ ]:
for s in strains: 
    for e in exp: 
        for i in mask: 
            print(f'Rscript mice.R {i} {filepath_common}{s}_{e}/{hl} {filepath_common}{s}_{e}/{dat}_{i}.csv')

In [21]:
import pandas as pd

# Example DataFrames
data_true = {
    'ID': ['id1', 'id2', 'id3'],
    'value': [10, 20, 30],
    'value2': [10, 20, 30]
}
data_pred = {
    'ID': ['id1', 'id2', 'id3'],
    'value': [10, 20, 30],
    'value2': [10, 21, 30]
}

df_true = pd.DataFrame(data_true)
df_pred = pd.DataFrame(data_pred)

In [22]:
import numpy as np 
true = np.array(df_true.drop('ID', axis=1))
pred = np.array(df_pred.drop('ID', axis=1))
true, pred


(array([[10, 10],
        [20, 20],
        [30, 30]]),
 array([[10, 10],
        [20, 21],
        [30, 30]]))

In [23]:
(true - pred)**2

array([[0, 0],
       [0, 1],
       [0, 0]])

In [24]:
#sum of squared differences
np.sum((true - pred)**2)

1